# Imort packages

In [11]:
import os
from natsort import natsorted
import pandas as pd
import numpy as np
import requests
from io import StringIO

# Query Examples

## od_by place

In [40]:
params = {"operation": "od_by_place",
       "odt_cube": "twitter_od_2020_county",
        "place": "12086",
       "begin": "01/01/2020",
       "end": "01/10/2020",
       "direction": "both"}

q = r'http://gis.cas.sc.edu/GeoAnalytics/REST'

r = requests.get(q, params=params)

In [43]:
df = pd.read_csv(StringIO(r.text), header=None, names=["place", "count"])
df

,place,count
0,5067,1
1,34025,4
2,26065,1
3,48329,2
4,12049,3
...,...,...
271,25025,19
272,17021,3
273,48453,9
274,24510,2


## od_daily_movement_by_place

In [47]:
params = {"operation": "od_daily_movement_by_place",
       "odt_cube": "twitter_od_2020_county",
        "place": "12086",
       "direction": "both"}

q = r'http://gis.cas.sc.edu/GeoAnalytics/REST'

r = requests.get(q, params=params)

In [49]:
df = pd.read_csv(StringIO(r.text), header=None, names=["date", "count"])
df

,date,count
0,2020-01-01,696
1,2020-01-02,546
2,2020-01-03,606
3,2020-01-04,526
4,2020-01-05,580
...,...,...
353,2020-12-26,292
354,2020-12-27,279
355,2020-12-28,315
356,2020-12-29,310


## od_daily_movement

In [52]:
params = {"operation": "od_daily_movement",
       "odt_cube": "twitter_world_country",
        "place": "12086",
       "begin": "11/01/2019",
       "end": "03/10/2020"}

q = r'http://gis.cas.sc.edu/GeoAnalytics/REST'

r = requests.get(q, params=params)

In [54]:
df = pd.read_csv(StringIO(r.text))
df

,place,date,intra_movement
0,AFG,2019-11-01,6
1,AFG,2019-11-02,5
2,AFG,2019-11-03,4
3,AFG,2019-11-04,8
4,AFG,2019-11-05,7
...,...,...,...
25917,ZWE,2020-03-06,38
25918,ZWE,2020-03-07,28
25919,ZWE,2020-03-08,26
25920,ZWE,2020-03-09,24


## od_extract_data

In [57]:
params = {"operation": "od_extract_data",
       "odt_cube": "twitter_od_2019_county",
       "begin": "01/01/2019" ,
       "end": "01/10/2019",
       "bbox": "20.19346,49.33148,-125.19531,-64.10156",
       "type": "aggregated"}

q = r'http://gis.cas.sc.edu/GeoAnalytics/REST'
r = requests.get(q, params=params)

In [58]:
df = pd.read_csv(StringIO(r.text))
df

,o_place,d_place,cnt,o_lat,o_lon,d_lat,d_lon
0,6085,8093,10,37.337,-121.891,38.998,-105.551
1,12005,36061,1,30.176,-85.694,40.752,-73.993
2,47093,21211,1,35.952,-83.925,38.134,-85.056
3,13045,13247,1,33.580,-85.069,33.664,-83.987
4,47127,1103,1,35.283,-86.374,34.607,-87.088
...,...,...,...,...,...,...,...
71588,47163,39117,1,36.512,-82.523,40.365,-82.669
71589,24033,54065,1,38.753,-76.992,39.449,-78.284
71590,21079,21021,1,37.612,-84.576,37.639,-84.774
71591,47019,48113,1,36.416,-82.155,32.918,-96.765


## od_download_url

In [70]:
params = {"operation": "od_download_url",
       "odt_cube": "twitter_od_2020_county",
       "begin": "01/01/2020" ,
       "end": "01/10/2020",
       "bbox": "20.19346,49.33148,-125.19531,-64.10156",
       "type": "daily"}

q = r'http://gis.cas.sc.edu/GeoAnalytics/REST'
r = requests.get(q, params=params)

In [75]:
download_url, count = r.text.split(",")
print("download_url:", download_url)
print("count:", count)

# download the response
file_name = download_url.split('/')[-1]
print("Downloading data...")
download_resp = requests.get(download_url, allow_redirects=True)
print("Saving data...")
f = open(file_name, 'w')
f.write(download_resp.text)
print("Done.")
df = pd.read_csv(StringIO(download_resp.text))
df

download_url: http://gis.cas.sc.edu/GeoAnalytics/download/daily_twitter_od_2020_county_01012020_01102020_1615433684004.csv
count: 154282
Saving data...
Done.


,o_place,d_place,year,month,day,cnt,o_lat,o_lon,d_lat,d_lon
0,49035,6111,2020,1,4,1,40.764,-111.823,34.267,-118.731
1,48113,29105,2020,1,6,1,32.820,-96.762,37.676,-92.661
2,17179,17107,2020,1,4,2,40.654,-89.504,40.126,-89.374
3,25025,12086,2020,1,10,2,42.314,-71.089,25.804,-80.301
4,5051,48095,2020,1,5,1,34.487,-93.021,31.169,-100.077
...,...,...,...,...,...,...,...,...,...,...
154277,39025,39027,2020,1,9,1,38.962,-84.064,39.299,-83.969
154278,26045,26045,2020,1,1,25,42.685,-84.724,42.688,-84.711
154279,54011,54019,2020,1,1,1,38.408,-82.440,37.986,-81.153
154280,42019,42003,2020,1,4,9,40.724,-80.036,40.503,-80.039
